In [140]:
import numpy as np
import pandas as pd
import csv
from collections import defaultdict
import networkx as nx
import json

In [141]:
df_C_raw = pd.read_excel("hierarchy_org_position_id.xlsx", sheet_name="CDIO_COO_DIV PRIVATE BANKING")
df_P_raw = pd.read_excel("hierarchy_org_position_id.xlsx", sheet_name="Private Banking")
df_OP = pd.read_excel("hierarchy_org_position_id.xlsx", sheet_name="Organizational Position")

df_Employees = pd.read_excel("Data.xlsx", sheet_name="Employees")
df_Domain = pd.read_excel("Data.xlsx", sheet_name="Knowledge structure")
df_Assessment = pd.read_excel("Data.xlsx", sheet_name="Knowledge assessment", na_values = "NA")
df_Organization = pd.read_excel("Data.xlsx", sheet_name="Organizational structure", na_values="NA")
df_Persona = pd.read_excel("Data.xlsx", sheet_name="Personas")

### Employment

#### Distribution of Age

In [142]:
df_Employees['AGE'].describe() 

count    65021.000000
mean        49.680273
std          8.618742
min         21.000000
25%         43.000000
50%         51.000000
75%         57.000000
max         71.000000
Name: AGE, dtype: float64

#### Counts for Sex

In [143]:
df_Employees['SEX'].value_counts()

F    33016
M    31691
Name: SEX, dtype: int64

#### Count Employee

In [144]:
len(df_Employees['ID_EMPLOYEE'].unique())

64998

In [145]:
len(df_Employees.index)

65021

#### There should be duplicated ID_EMPLOYEE

In [146]:
a_list = df_Employees.ID_EMPLOYEE.tolist()
a_set = set(a_list)
contains_duplicates = len(a_list) != len(a_set)

list1=[]
duplicated_ID = []
for i in a_list:
    if i not in list1:
        list1.append(i)
    else:
        duplicated_ID.append(i)
duplicated_ID

[8503,
 10049,
 11113,
 30411,
 33046,
 33051,
 33052,
 33092,
 33928,
 59459,
 60997,
 61039,
 61241,
 61259,
 61399,
 62073,
 62090,
 62114,
 62116,
 62124,
 62163,
 62289,
 62350]

#### Duplicated ID have different Orgnization_Position

In [147]:
df_Employees.loc[df_Employees['ID_EMPLOYEE'].isin(duplicated_ID)]

,ID_EMPLOYEE,SEX,AGE,ID_FRAME,FLAG_RESPONSABILE,ID_ORGANIZATIONAL_POSITION
8421,8503,M,62,QD/4,#,4593
8422,8503,M,62,QD/4,#,5051
9955,10049,M,57,QD/4,#,4596
9956,10049,M,57,QD/4,#,5107
11006,11113,M,51,QD/2,#,4593
11007,11113,M,51,QD/2,#,5051
29772,30411,M,63,QD/1,#,4607
29773,30411,M,63,QD/1,#,6145
32405,33046,M,63,QD/4,#,4607
32406,33046,M,63,QD/4,#,6145


### Assessment

In [148]:
df_Assessment

,ID_EMPLOYEE,ID_MANAGER,ID_KNOWLEDGE_DOMAIN,SCORE_AUTODETECTION,SCORE_VALIDATION
0,63374,767,C05_1D01,1,NaN
1,39838,42166,C11D02,1,NaN
2,27198,8078,C15D03,1,1.0
3,28622,20863,C23D02,1,NaN
4,20494,25046,C28D04,1,NaN
...,...,...,...,...,...
1048570,34556,35036,C35_1D03,1,NaN
1048571,265,18190,C07D01,1,NaN
1048572,3117,67408,C41D01,1,NaN
1048573,63649,63704,C32D03,1,1.0


b_list = df_Assessment.ID_EMPLOYEE.tolist()
b_set = set(b_list)
contains_duplicates = len(b_list) != len(b_set)

list2=[]
duplicated_Assessment = []
for i in b_list:
    if i not in list2:
        list2.append(i)
    else:
        duplicated_Assessment.append(i)
duplicated_Assessment


#### Duplicated Employee has duplicated Assessment. Assessments are the same

In [149]:
duplicated_Assessment == duplicated_ID

False

In [150]:
df_Assessment.loc[df_Assessment['ID_EMPLOYEE'].isin(duplicated_Assessment)]

,ID_EMPLOYEE,ID_MANAGER,ID_KNOWLEDGE_DOMAIN,SCORE_AUTODETECTION,SCORE_VALIDATION
0,63374,767,C05_1D01,1,NaN
1,39838,42166,C11D02,1,NaN
2,27198,8078,C15D03,1,1.0
3,28622,20863,C23D02,1,NaN
4,20494,25046,C28D04,1,NaN
...,...,...,...,...,...
1048570,34556,35036,C35_1D03,1,NaN
1048571,265,18190,C07D01,1,NaN
1048572,3117,67408,C41D01,1,NaN
1048573,63649,63704,C32D03,1,1.0


#### distribution of SCORE_AUTODETECTION

In [151]:
df_Assessment['SCORE_AUTODETECTION'].value_counts()

1    864221
2    113101
3     46071
4     19264
5      5918
Name: SCORE_AUTODETECTION, dtype: int64

#### A large number of Employees don't have SCORE_VALIDATION in specific Knowledge domain. Some knowledge domains have large missing self-score.

In [152]:
df_NoSelfScore= df_Assessment.loc[df_Assessment['SCORE_VALIDATION'].isna()]
df_NoSelfScore['ID_KNOWLEDGE_DOMAIN'].value_counts()

C19_1D02    7677
C35_2D01    7677
C35_2D02    7676
C24D02_1    7645
C19_1D01    7634
            ... 
C38D02       522
C16_1D13     511
C40D01       342
C37D01       164
C38D01        27
Name: ID_KNOWLEDGE_DOMAIN, Length: 130, dtype: int64

In [153]:
len(df_NoSelfScore)

649416

In [154]:
df_With_Self_Score = df_Assessment[df_Assessment['SCORE_VALIDATION'].notna()]

In [155]:
df_With_Self_Score

,ID_EMPLOYEE,ID_MANAGER,ID_KNOWLEDGE_DOMAIN,SCORE_AUTODETECTION,SCORE_VALIDATION
2,27198,8078,C15D03,1,1.0
11,54759,43553,C29D02,1,1.0
16,8217,3976,C11D03,1,1.0
20,26640,28488,C41D01,1,1.0
26,44864,4853,C30_1D05,1,1.0
...,...,...,...,...,...
1048566,17752,17858,C01D04,1,1.0
1048567,44217,1903,C03D01,5,5.0
1048569,16261,63573,C16_1D13,1,1.0
1048573,63649,63704,C32D03,1,1.0


#### 86.9 percent of self score match the manager score. 

In [156]:
(df_With_Self_Score['SCORE_AUTODETECTION'] - df_With_Self_Score['SCORE_VALIDATION']).value_counts(normalize = True)

 0.0    0.868949
 1.0    0.050125
-1.0    0.046603
 2.0    0.013240
-2.0    0.010800
 3.0    0.005146
-3.0    0.002656
 4.0    0.002017
-4.0    0.000463
dtype: float64

### Knowledge Domain 

In [157]:
df_Domain.loc[df_Domain["DESCR_KNOWLEDGE_DOMAIN"].isnull()]

,ID_KNOWLEDGE_DOMAIN,DESCR_KNOWLEDGE_DOMAIN,ID_KNOWLEDGE_AREA,DESCR_KNOWLEDGE_AREA


In [158]:
df_Domain.loc[df_Domain["DESCR_KNOWLEDGE_AREA"].isnull()]

,ID_KNOWLEDGE_DOMAIN,DESCR_KNOWLEDGE_DOMAIN,ID_KNOWLEDGE_AREA,DESCR_KNOWLEDGE_AREA
49,C01D03,Segmento Imprese,NaN,NaN
85,C32D06,Relazioni con i Regolatori,NaN,NaN


In [160]:
len(df_Domain['ID_KNOWLEDGE_DOMAIN'].unique())
# no duplicated index
#len(df_Domain['ID_KNOWLEDGE_DOMAIN'].unique()) = len(df_Domain.index)

130

In [161]:
len(df_Domain['ID_KNOWLEDGE_AREA'].unique())

43

In [162]:
df_Areas = df_Domain.groupby(["ID_KNOWLEDGE_AREA"])

In [163]:
df_Areas.first()

,ID_KNOWLEDGE_DOMAIN,DESCR_KNOWLEDGE_DOMAIN,DESCR_KNOWLEDGE_AREA
ID_KNOWLEDGE_AREA,,,
AREAC001,C01D01,Segmento Retail,Conoscenza Segmenti
AREAC002,C02D01,Customer experience management,Customer Relationship & Development
AREAC003,C03D01,Operativit√† di sportello,Conoscenza di Prodotti e Servizi bancari
AREAC004,C04D02,Trade Finance,Transaction Banking
AREAC005_1,C05_1D03,Prodotti previdenziali,Conoscenza di Prodotti e Servizi Assicurativi
AREAC006,C06D04,Accesso programmi di finanziamento,Conoscenza di Prodotti e Servizi non bancari
AREAC007,C07D03,Monitoraggio ciclo di vita prodotti e servizi,Product & Service development
AREAC008,C08D01,Trend di mercato e posizionamento competitivo,Business Development
AREAC009,C09D01,Corporate Finance,Credito e Finanza d'Impresa


### Create Network Data

In [164]:
df_C_raw = pd.read_excel("hierarchy_org_position_id.xlsx", sheet_name="CDIO_COO_DIV PRIVATE BANKING")
df_P_raw = pd.read_excel("hierarchy_org_position_id.xlsx", sheet_name="Private Banking")

In [165]:
df_C = df_C_raw[["UOG1_F","UOG2_F","UOG3_F","UOG4_F","UOG5_F","UOG6_F"]]
df_P = df_P_raw[["UOG1_F","UOG2_F","UOG3_F","UOG4_F","UOG5_F","UOG6_F"]]

In [196]:
# extract all org id
org_list = []
for i in df_C:
    org_list.append(df_C[i].tolist())

for i in df_P:
    org_list.append(df_P[i].tolist())

# flattern list
flat_list = []
for sublist in org_list:
    for item in sublist:
        flat_list.append(item)
        
org_id = np.unique(np.array(flat_list)).tolist()
org_id.remove("-")
org_id = list(map(int, org_id))

#### Extract all the unique orgnization_positions(from Private bankings) intended to investigate from the whole employees data

In [197]:
org_id

[1,
 10,
 100,
 10012,
 10019,
 10020,
 10036,
 10048,
 10049,
 10056,
 10077,
 10079,
 101,
 10112,
 10113,
 10115,
 10119,
 10120,
 102,
 103,
 104,
 105,
 10571,
 10572,
 10573,
 10574,
 10575,
 10576,
 10577,
 10578,
 10579,
 10580,
 10581,
 10582,
 10583,
 10584,
 10585,
 106,
 107,
 10730,
 10731,
 10732,
 10733,
 10734,
 10735,
 10736,
 10737,
 10738,
 10739,
 10740,
 10741,
 10742,
 10743,
 10744,
 10745,
 10746,
 10747,
 10748,
 10749,
 10750,
 10751,
 10752,
 10753,
 10754,
 10755,
 10756,
 10757,
 10758,
 10759,
 10760,
 10761,
 10762,
 10763,
 10764,
 10765,
 10766,
 10767,
 10768,
 10769,
 10770,
 10771,
 10772,
 10773,
 10774,
 10775,
 10776,
 10777,
 10778,
 10779,
 10780,
 10781,
 10782,
 10783,
 10784,
 10785,
 10786,
 10787,
 10788,
 10789,
 10790,
 10791,
 10792,
 10793,
 10794,
 10795,
 10796,
 10797,
 10798,
 10799,
 108,
 10800,
 10801,
 10802,
 10803,
 10804,
 10805,
 10806,
 10807,
 10808,
 10809,
 10810,
 10811,
 10812,
 10813,
 10814,
 10815,
 10816,
 10817,
 

### Filter the Employees that have the orgnization in the dataset

In [198]:
# filter employee ID by org ID
df_name = pd.merge(df_Employees, df_Organization, left_on = "ID_ORGANIZATIONAL_POSITION", 
                   right_on = "ID_ORGANIZATIONAL_POSITION")
df_name = df_name.loc[df_name["ID_ORGANIZATIONAL_POSITION"].isin(org_id)]

#### No more duplicated EmployeeID

In [201]:
len(df_name['ID_EMPLOYEE'].unique()) == len(df_name.index)

True

In [202]:
df_assess = pd.merge(df_Assessment, df_Domain, left_on = "ID_KNOWLEDGE_DOMAIN", 
                         right_on = "ID_KNOWLEDGE_DOMAIN")

In [203]:
# filter employee assessment by org ID
df = pd.merge(df_name, df_assess, left_on = "ID_EMPLOYEE",
             right_on = "ID_EMPLOYEE")
df = df.loc[df["ID_ORGANIZATIONAL_POSITION"].isin(org_id)]


In [206]:
df

,ID_EMPLOYEE,SEX,AGE,ID_FRAME,FLAG_RESPONSABILE,ID_ORGANIZATIONAL_POSITION,PROVINE,REGION,ID_PERSONAS,ID_MANAGER,ID_KNOWLEDGE_DOMAIN,SCORE_AUTODETECTION,SCORE_VALIDATION,DESCR_KNOWLEDGE_DOMAIN,ID_KNOWLEDGE_AREA,DESCR_KNOWLEDGE_AREA
0,6,F,52,QD/4,X,55,Milano,Lombardia,P00030,13080,C05_1D01,1,1.0,Prodotti danni,AREAC005_1,Conoscenza di Prodotti e Servizi Assicurativi
1,6,F,52,QD/4,X,55,Milano,Lombardia,P00030,13080,C11D01,1,1.0,Istruttoria e concessione del credito,AREAC011,Analisi e Gestione del credito
2,6,F,52,QD/4,X,55,Milano,Lombardia,P00030,13080,C34_1D03,1,NaN,Gestione servizi a supporto,AREAC034_1,Property
3,6,F,52,QD/4,X,55,Milano,Lombardia,P00030,13080,C12D01,4,NaN,Sistema finanziario,AREAC012,Capital Markets
4,6,F,52,QD/4,X,55,Milano,Lombardia,P00030,13080,C23D01,1,NaN,Analisi statistiche,AREAC023,Modelli quantitativi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160730,68363,M,49,DR/0,X,10731,Milano,Lombardia,P00036,14353,C30_1D01,1,NaN,Communication Plan,AREAC030_1,Comunicazione
160731,68363,M,49,DR/0,X,10731,Milano,Lombardia,P00036,14353,C06D03,1,NaN,Servizi professionali alle imprese,AREAC006,Conoscenza di Prodotti e Servizi non bancari
160732,68363,M,49,DR/0,X,10731,Milano,Lombardia,P00036,14353,C38D01,1,1.0,Program & Project Management,AREAC038,Program & Project Management
160733,68363,M,49,DR/0,X,10731,Milano,Lombardia,P00036,14353,C04D02,1,1.0,Trade Finance,AREAC004,Transaction Banking


In [208]:
df.duplicated().sum()

31

In [209]:
df.loc[df.duplicated(), :]

,ID_EMPLOYEE,SEX,AGE,ID_FRAME,FLAG_RESPONSABILE,ID_ORGANIZATIONAL_POSITION,PROVINE,REGION,ID_PERSONAS,ID_MANAGER,ID_KNOWLEDGE_DOMAIN,SCORE_AUTODETECTION,SCORE_VALIDATION,DESCR_KNOWLEDGE_DOMAIN,ID_KNOWLEDGE_AREA,DESCR_KNOWLEDGE_AREA
1086,10863,F,61,DR/0,X,8275,Milano,Lombardia,P00019,776,C02D03,1,1.0,Customer Development,AREAC002,Customer Relationship & Development
28602,15727,M,48,DR/0,X,8239,Milano,Lombardia,P00019,776,C16_1D06,1,NaN,Normativa specialistica,AREAC016_1,Normativa e Compliance
39171,13862,M,48,QD/4,X,8247,Milano,Lombardia,P00019,15223,C28D04,1,NaN,Sistemi retributivi,AREAC028,Risorse Umane
39183,13862,M,48,QD/4,X,8247,Milano,Lombardia,P00019,15223,C36D02,1,1.0,Service Design,AREAC036,Digital
52554,5425,M,49,DR/0,X,8289,Milano,Lombardia,P00019,15545,C41D01,1,1.0,Relazioni internazionali,AREAC041,International Network
64703,13783,M,63,DR/0,X,8287,Parma,Emilia Romagna,P00019,15093,C29D02,2,NaN,Processi e procedure aziendali,AREAC029,Modelli e processi organizzativi
64714,13783,M,63,DR/0,X,8287,Parma,Emilia Romagna,P00019,15093,C26D03,1,NaN,Gestione e manutenzione IT,AREAC026,Architettura IT e software development
64722,13783,M,63,DR/0,X,8287,Parma,Emilia Romagna,P00019,15093,C22D02,2,2.0,Data Analysis,AREAC022,Data governance
82040,13428,M,59,QD/4,X,8292,Padova,Veneto,P00019,62825,C14D02,1,NaN,ALM & Capital Management,AREAC014,Funding
82377,13584,F,47,QD/4,X,8277,Parma,Emilia Romagna,P00019,14847,C28D03,1,NaN,Gestione e Valutazione,AREAC028,Risorse Umane


### Create Network

In [210]:
df_nodes = pd.DataFrame(df["ID_EMPLOYEE"].unique())
df_nodes = df_nodes.rename(columns={0:"Node"})
df_nodes_domain = pd.DataFrame(df["DESCR_KNOWLEDGE_AREA"].unique())
df_nodes_domain = df_nodes_domain.rename(columns={0:"Node"})
df_nodes_domain = df_nodes_domain.dropna()
df_nodes = df_nodes.append(df_nodes_domain)

In [211]:
nodes = [n for n in df_nodes["Node"].to_numpy()][0:]

In [212]:
len(nodes)

9963

#### Create Edge

In [213]:
df_edges = df.groupby(["ID_EMPLOYEE","DESCR_KNOWLEDGE_AREA"]).mean()

In [214]:
df_edges = df_edges.reset_index(["ID_EMPLOYEE","DESCR_KNOWLEDGE_AREA"])
df_edges["SCORE_AVG"] = df_edges[["SCORE_AUTODETECTION","SCORE_VALIDATION"]].mean(axis=1)
df_edges = df_edges[["ID_EMPLOYEE","DESCR_KNOWLEDGE_AREA","SCORE_AVG"]]

In [215]:
weighted_edges = [tuple(n) for n in df_edges.to_numpy()]

In [216]:
len(df_edges)

129464